## 1. 환경 설정

In [ ]:
# GPU 확인
!nvidia-smi

In [ ]:
# 저장소 클론 (기존 디렉토리 있으면 삭제 후 재클론)
!rm -rf -prompt-firewall
!git clone https://github.com/mlnyx/-prompt-firewall.git
%cd ./-prompt-firewall

In [ ]:
# 필수 패키지 설치
!pip install -q transformers torch sentence-transformers pyyaml pandas tqdm accelerate bitsandbytes

## 2. Hugging Face 토큰 설정 (Llama 3 접근용)

1. https://huggingface.co/settings/tokens 에서 토큰 생성
2. https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct 에서 모델 접근 권한 요청
3. 아래 셀에 토큰 입력

In [ ]:
from huggingface_hub import login

# 여기에 토큰 입력
HF_TOKEN = "hf_..."
login(token=HF_TOKEN)

## 3. 평가 실행

**참고:** 
- Llama 3 접근 승인 전: Stage 1-2만 실행됩니다
- Llama 3 접근 승인 후: Stage 1-3 전체 실행됩니다

In [ ]:
# Stage 1-2 평가 (Llama 3 승인 전에도 실행 가능)
!python evaluate.py --no-llm --sample 512

In [ ]:
import pandas as pd

# Stage 2 결과 로드
jailbreak_df = pd.read_csv('data/stage2_rewrites(jailbreak).txt')
benign_df = pd.read_csv('data/stage2_rewrites(benign).txt')

print(f"Jailbreak: {len(jailbreak_df)}개")
print(f"Benign: {len(benign_df)}개")

# Jailbreak: score 낮은 순 (애매한 악성)
jailbreak_samples = jailbreak_df.nsmallest(256, 'score')

# Benign: score 높은 순 (애매한 정상)
benign_samples = benign_df.nlargest(256, 'score')

# 샘플 결합
samples = pd.concat([jailbreak_samples, benign_samples])

# 샘플 저장
samples.to_csv('stage3_samples.csv', index=False)
print(f"\n✓ Stage 3 샘플 512개 저장: stage3_samples.csv")
print(f"  - Jailbreak (low score): {len(jailbreak_samples)}개")
print(f"  - Benign (high score): {len(benign_samples)}개")

### 3-1. Stage 2 결과 샘플링 (선택사항)

다운로드한 `stage2_rewrites(jailbreak).txt`와 `stage2_rewrites(benign).txt`에서 
극단적인 케이스를 선별하여 Stage 3 테스트에 사용할 수 있습니다:
- **Jailbreak**: score가 낮은 것 (Stage 2가 애매하게 판단한 악성)
- **Benign**: score가 높은 것 (Stage 2가 애매하게 판단한 정상)

## 4. Stage 3 테스트 (샘플링된 데이터 사용)

In [ ]:
# Stage 3 평가 (샘플링된 512개만 사용)
# Llama 3 접근 승인 완료 후 실행하세요
# !python -c "
# from tester_framework.core import Population
# from tester_framework.runners import Stage3LocalRunner
# from tester_framework.orchestrator import Tester
# 
# pop = Population()
# pop.create_population_from_file('stage3_samples.csv')
# runner = Stage3LocalRunner(use_local_llm=True)
# tester = Tester(pop, runner)
# results = tester.run_all()
# pop.save_to_csv('stage3_results.csv')
# print(f'완료: {len(results)}개')
# "

In [ ]:
## 5. 결과 다운로드

In [ ]:
from google.colab import files

# Stage 2 결과 다운로드
!ls -lh stage2_rewrites.txt && files.download('stage2_rewrites.txt')

# Stage 2 점수 다운로드
!ls -lh data/s2_all_scores.csv && files.download('data/s2_all_scores.csv')